In [1]:
import pandas as pd
import pickle
import re
import string
from compound_split import doc_split

from tqdm.notebook import tqdm as tq

In [2]:
import os
path = '../../data/epo_corpus_all_years/'
for x in os.walk(path):
    _, _, files = x
    break

files

['epo_2013_b60_df.gzip',
 'epo_2015_b60_df.gzip',
 'epo_2019_b60_df.gzip',
 'Spell Error Dataset.txt',
 'Noun chunks gen',
 'epo_2018_b60_df.gzip',
 'epo_2010_b60_df.gzip',
 'epo_2012_b60_df.gzip',
 'epo_2020_b60_df.gzip',
 'epo_2011_b60_df.gzip',
 'epo_2014_b60_df.gzip',
 'epo_2016_b60_df.gzip',
 'epo_2017_b60_df.gzip']

In [3]:
# get the B60 patent dataframes of all years
epo_20xx = []
for file in files:
    if 'gzip' in file:
        epo_20xx.append(pd.read_pickle('%s/%s'%(path, file), compression='gzip'))

In [4]:
len(epo_20xx)

11

In [5]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [6]:
import spacy
from spacy.matcher import Matcher
nlp_en = spacy.load("en_core_web_lg")
nlp_de = spacy.load("de_core_news_lg")

In [237]:
def pre_process_claim(claim, lang):
    # add space between punctuation and the word
    # ,wherein --> , wherein
    # claim = re.sub(r'([,;./:])([a-zA-Z])', r' \2', claim)
    # claim = re.sub(r'([a-zA-Z])([,;./:])', r'\1 \2', claim)

    # remove punctuations except for :;
    claim = re.sub(r'[!"#$%&\()*+/<=>?@\[\\\\\]^_`{|}~]', '', claim)
    
    c_s = claim.split(' ')
    
    if lang == 'en':
        # replace claims references to [CLAIMS]
        # claims 1 to 2 --> [CLAIMS]
        # claim = re.sub(r'[c|C]laims\s\d{1,}\s[a-z]{1,}\s\d{1,}', '', claim)
        # # claim 1 to 2 --> [CLAIMS]
        # claim = re.sub(r'[c|C]laim\s\d{1,}\s[a-z]{1,}\s\d{1,}', '', claim)
        # # claim 1 --> [CLAIMS]
        # claim = re.sub(r'[c|C]laim\s\d{1,}', '', claim)
        # claim = re.sub(r'andan', 'and an', claim)
        # claim = re.sub(r'[0-9]{1,}', 'NUM', claim)
        for s in c_s:
            if s in incorrect_words_en.keys():
                # print(s)
                claim = re.sub(s, incorrect_words_en[s], claim)

    elif lang == 'de':
        # replace claims references to [CLAIMS]
        # claims 1 to 2 --> [CLAIMS]
        # claim = re.sub(r'Anspr[u|ü]ch[e|en]{0,}\s\d{1,}\s[a-z]{1,}\s\d{1,}', '', claim)
        # # claim 1 to 2 --> [CLAIMS]
        # # i = re.sub(r'Ansprüche\s\d{1,}\s[a-z]{1,}\s\d{1,}', r'[CLAIMS]', i)
        # # i = re.sub(r'Ansprüchen\s\d{1,}\s[a-z]{1,}\s\d{1,}', r'[CLAIMS]', i)
        # # claim 1 --> [CLAIMS]
        # claim = re.sub(r'Anspruch\s\d{1,}', '', claim)
        
        # split compound German words
        # claim = doc_split.doc_split(claim)
        # claim = re.sub('·', ' ', claim)
        
        for s in c_s:
            if s in incorrect_words_de.keys():
                # print(s)
                claim = re.sub(s, incorrect_words_de[s], claim)        
    
    # replace groups of numbers with NUM
    # eg. 100, 200 --> NUM
    claim = re.sub(r'([0-9]{1,},{1,}\s{0,})+[0-9]{1,}', 'NUM ', claim)
    claim = re.sub(r'([a-zA-Z\']{0,}[0-9]{1,}[,;:]{1,}\s{0,})+[a-zA-Z\']{0,}[0-9]{1,}', 'NUM ', claim)
    # replace single number to NUM
    # eg. 1 --> NUM
    claim = re.sub(r'[0-9]{1,}', 'NUM', claim)        
    
    # replace contents within parantheses as [ABBRV]
    claim = re.sub(r'\(.+?\)', '', claim)
    
    # replace newline characters with space
    claim = re.sub(r'\n', ' ', claim)
    
    # replace punctuations with space
    # claim = re.sub(r'(\w)([!"#$%&\'()*+,-./:;<=>?@\[\\\\\]^_`{|}~])(\w)', '\1 \3', claim)
    
    claim = re.sub(r'([!"#$%&\'()*+,-./:;<=>?@\[\\\\\]^_`{|}~])', r' \1 ', claim)
    
    # claim = re.sub(r'([!"#$%&\'()*+,-./:;<=>?@\[\\\\\]^`{|}~])', ' ', claim)
    claim = re.sub(r'([a-zA-Z])-\s', r'\1 -', claim)
    claim = re.sub(r'[^a-zA-Z][-\']', r' ', claim)
    
    claim = re.sub(r'\(([a-zA-Z\s]{0,}NUM[a-zA-Z\s]{0,})+\)', 'NUM', claim)
    
    # replace double spaces with single space
    claim = re.sub(r'\s{1,}', ' ', claim)    
    
    return claim

In [238]:
claim = epo_20xx[0].en_claims[189][0]
claim

'A cleaning apparatus (1) for a vehicle lamp comprising:\na nozzle portion (4) having a jet nozzle (16) for jetting a cleaning liquid;a nozzle cover (6) for covering the nozzle portion (4) from a front side;a piston (3) connected to the nozzle portion (4) and integrally movable with the nozzle portion (4) and the nozzle cover (6) in a movement direction;a cylinder (2) movably supporting the piston (3);a bracket (22) that includes an arm portion (24) located in abutment with or in proximity with a rear surface of the nozzle cover (6), and regulates an inclination of the nozzle cover (6) with respect to the movement direction; anda holder (5) provided between the nozzle cover (6) and the nozzle portion (4) for connecting the nozzle cover (6) and the nozzle portion (4) and integrally movable with the piston (3) in the movement direction;characterized bya leg portion (23) that is provided in the bracket (22) and regulates an inclination of the holder (5) with respect to the movement direct

In [239]:
pre = pre_process_claim(claim, lang='en')
pre

'A cleaning apparatus NUM for a vehicle lamp comprising : a nozzle portion NUM having a jet nozzle NUM for jetting a cleaning liquid ; a nozzle cover NUM for covering the nozzle portion NUM from a front side ; a piston NUM connected to the nozzle portion NUM and integrally movable with the nozzle portion NUM and the nozzle cover NUM in a movement direction ; a cylinder NUM movably supporting the piston NUM ; a bracket NUM that includes an arm portion NUM located in abutment with or in proximity with a rear surface of the nozzle cover NUM , and regulates an inclination of the nozzle cover NUM with respect to the movement direction ; and a holder NUM provided between the nozzle cover NUM and the nozzle portion NUM for connecting the nozzle cover NUM and the nozzle portion NUM and integrally movable with the piston NUM in the movement direction ; characterized by a leg portion NUM that is provided in the bracket NUM and regulates an inclination of the holder NUM with respect to the moveme

In [227]:
find_and_replace_ncs(pre, lang='en')

comp_ tire tread
comp_ first set
comp_ second set


'A tire_tread comprising a structure NUM , the structure NUM comprising a radially extending pedestal NUM , a first_set of voids NUM extending radially through the pedestal NUM , a second_set of voids NUM extending radially through the pedestal NUM and a radially extending void ring NUM circumscribing the pedestal NUM , characterized in that'

In [70]:
import sys
sys.path.append('../')
from util import corpus, tokenize, util, alignment, word2vec, retrieval

In [71]:
def add_space_between_words(key, word):
    # correct words starting with key word and no space
    corr_word = ''
    if re.search(r'^%s'%key, word):
        if '-' in word:
            corr_word = ''
            pass
        elif not wn.synsets(word):
            corr_word = re.sub(r'^%s'%key, '%s '%key, word)
            # print(word, '-->', corr_word)
    return corr_word

In [72]:
'''
and, 
whereas,
wherein,
least,
that,
by,
particular,
there,
characterized,

'''

'''
und, 
wohingegen,
wherein,
worin,
dass -- not sure,
von,
characterized,
gekennzeichnet,
der
'''

'\nund, \nwohingegen,\nwherein,\nworin,\ndass -- not sure,\nvon,\ncharacterized,\ngekennzeichnet,\nder\n'

In [73]:
EN_CORR_KEYS = [\
                'and', 
                'whereas',
                'wherein',
                'least',
                'that',
                'by',
                'particular',
                'there',
                'characterized',
               ]

DE_CORR_KEYS = [\
                'und', 
                'wohingegen',
                'wherein',
                'worin',
                'dass',
                'von',
                'characterized',
                'gekennzeichnet',
                'der'
               ]


In [74]:
incorrect_words_en = {}
for word in tq(epo_corpus.vocab_en):
    for key in EN_CORR_KEYS:
        corr_word = add_space_between_words(key, word)
        if corr_word != '':
            # print(corr_word)
            incorrect_words_en[word] = corr_word

In [75]:
incorrect_words_de = {}
for word in tq(epo_corpus.vocab_de):
    for key in DE_CORR_KEYS:
        corr_word = add_space_between_words(key, word)
        if corr_word != '':
            incorrect_words_de[word] = corr_word

In [76]:
len(incorrect_words_en), list(incorrect_words_en.keys())[:10]

(7739,
 ['androlling',
  'andrear',
  'that18a',
  'thatend',
  'thatpfcappl',
  'thatfb1',
  'byeach',
  'thatreceive',
  'thatincision',
  'thategoegoegoegoegoegoegoegoego'])

In [77]:
len(incorrect_words_de), list(incorrect_words_de.keys())[:10]

(6653,
 ['dassθvv',
  'undherausführen',
  'dererzeugen',
  'undlernen',
  'dassupc\u2062h\u2062arg\u2062em11pd\u2062ec\u2062h\u2062arg\u2062em11pa\u2062c\u2062h\u2062a\u2062tm14preventem14plocalconsmgtpc\u2062h\u2062arg\u2062em11pd\u2062e\u2062c\u2062h\u2062arg\u2062em11pachatm14preventem14plocalconsm8',
  'unde2',
  'undkopplungsabschnitte',
  'dassimaximax',
  'undeingriffssteuerungsmittel',
  'undthermoformen'])

## Find noun chunks

In [343]:
WORDS_TO_OMIT = ['NUM', 'claim', 'Claim', 
                'anspr', 'Anspr']
def find_and_replace_ncs(claim, lang):    
    nlp = nlp_en if lang=='en' else nlp_de
    doc = nlp(claim)
    for nc in doc.noun_chunks:
        comp_ = []
        for token in nc:
            if token.pos_ not in ['DET', 'ADV', 'NUM', 'CCONJ'] and \
               token.dep_ not in ['poss'] and \
               token.text not in ['said', 'more', 'following', 'comprising'] and \
               not token.is_punct:
                comp_.append(token.text)
        comp_ = ' '.join(comp_)
        comp_ = re.sub(r'(\s\s)+', '', comp_)
        if comp_:
            if len(comp_.split(' ')) > 1:
                omit_flag = False
                for word in WORDS_TO_OMIT:
                    if word in comp_:
                        omit_flag = True
                        break
                if not omit_flag:
                    # rep = re.sub('-', '', comp_)
                    rep = re.sub(r'\s{1,}', '_', comp_)
                    # print('comp_', comp_)
                    claim = re.sub(comp_, rep, claim)
    return claim

## Pre-process claims

In [331]:
def preprocess_claims(epo_df):
    pre_claims = []
    for name, ipcrs, titles, en_claims, de_claims in zip(tq(epo_df.name),
                                                         epo_df.ipcrs,
                                                         epo_df.titles,
                                                         epo_df.en_claims,
                                                         epo_df.de_claims):
        en_claims_proc = []
        de_claims_proc = []
        
        for en, de in zip(en_claims, de_claims):
            if len(en.split(' ')) <= 13:
                continue

            # ---- EN ---- #
            # to pre process the claim
            pre = pre_process_claim(en, lang='en')
            # to add underscores to noun chunks
            pre = find_and_replace_ncs(pre, lang='en')
            # finally, append the claim
            en_claims_proc.append(pre)
            
            # ---- DE ---- #
            # to pre process the claim
            pre = pre_process_claim(de, lang='de')
            # to add underscores to noun chunks
            pre = find_and_replace_ncs(pre, lang='de')
            # finally, append the claim
            de_claims_proc.append(pre)

        details = {
            'name': name,
            'iprcs': ipcrs,
            'titles': titles,
            'en_claims': en_claims_proc,
            'de_claims': de_claims_proc
        }
        pre_claims.append(details)
    return pre_claims

In [332]:
df_ = epo_20xx[0][:1000]
df_

,name,ipcrs,titles,en_claims,de_claims,fr_claims
0,EP03703859NWB1.xml,"[B60L 11/12 20060101AFI20121031BHEP, B60L 11/1...",de\tMODULARER LEISTUNGSWANDLER MIT FLÜSSIGGEKÜ...,[A modular power converter comprising:\na ther...,"[Modularer Leistungswandler, der Folgendes umf...",[Convertisseur électrique modulaire comprenant...
1,EP04021666NWB1.xml,"[F16H 61/12 20100101AFI20121030BHEP, B60W 50/0...",de\tVerfahren zur Steuerung eines Kraftfahrzeu...,[Method for controlling a motor vehicle drive ...,[Verfahren zur Steuerung eines Kraftfahrzeug-A...,[Procédé de commande d'un système de traction ...
2,EP03798697NWB1.xml,"[B29C 44/06 20060101AFI20121002BHEP, B60R 13/0...",de\tVERBESSERTE FAHRZEUGBODENBELÄGE UND VERFAH...,[A method of producing a vehicle interior floo...,[Verfahren zum Herstellen einer Innenbodenabde...,[Procédé de production d'un revêtement de plan...
3,EP03815372NWB1.xml,"[B60J 5/06 20060101AFI20040810BHEP, B60J 5/04 ...",de\tSCHWENKSCHIEBETÜR FÜR FAHRZEUGE\ten\tPIVOT...,"[Plug door for vehicles, in particular for rai...","[Schwenkschiebetür für Fahrzeuge, insbesondere...",[Porte louvoyante coulissante pour des véhicul...
4,EP05017320NWB1.xml,"[B60Q 3/04 20060101AFI20051116BHEP, B60K 37/00...",de\tKfz-Innenbeleuchtungsstruktur\ten\tVehicle...,[A vehicle interior illumination structure com...,"[Fahrzeuginnenraum-Beleuchtungsstruktur, welch...",[Structure d'éclairage intérieur de véhicule c...
...,...,...,...,...,...,...
995,EP08791706NWB1.xml,"[B60C 1/00 20060101AFI20121016BHEP, C08L 23/22...",de\tLuftreifen mit Schutzschicht\ten\tPneumati...,[A pneumatic tire using a thermoplastic elasto...,[Luftreifen unter Verwendung eines thermoplast...,[Bandage pneumatique utilisant un élastomère t...
996,EP08792532NWB1.xml,"[B60W 10/00 20060101AFI20090325BHEP, B60K 6/22...",de\tHYBRIDFAHRZEUG\ten\tHYBRID VEHICLE\tfr\tVÉ...,[A hybrid vehicle running by consuming a first...,"[Hybridfahrzeug, das unter Verwendung einer er...",[Véhicule hybride se déplaçant en consommant u...
997,EP08794110NWB1.xml,"[B60T 1/08 20060101AFI20120919BHEP, B60W 10/18...",de\tBREMSSYSTEM UND FAHRZEUG MIT EINEM BREMSSY...,[A vehicle (10) having a powertrain and brake ...,[Fahrzeug (10) mit einem Antriebsstrang und ei...,[Véhicule (10) ayant une transmission et un sy...
998,EP08801238NWB1.xml,"[B64C 25/36 20060101AFI20120926BHEP, F16C 19/5...",de\tRADLAGERUNG EINES FLUGZEUGFAHRWERKS\ten\tW...,[Wheel bearing (1) of an aircraft landing gear...,"[Radlagerung (1) eines Flugzeugfahrwerks,dadur...",[Système de palier de roue (1) d'un train d'at...


In [333]:
epo_10_20_proc = []
epo_10_20_proc.extend(preprocess_claims(df_))

epo_10_20_proc_df = pd.DataFrame(epo_10_20_proc)
epo_10_20_proc_df

,name,iprcs,titles,en_claims,de_claims
0,EP03703859NWB1.xml,"[B60L 11/12 20060101AFI20121031BHEP, B60L 11/1...",de\tMODULARER LEISTUNGSWANDLER MIT FLÜSSIGGEKÜ...,[A modular power converter comprising : a ther...,"[Modularer_Leistungswandler , der Folgendes um..."
1,EP04021666NWB1.xml,"[F16H 61/12 20100101AFI20121030BHEP, B60W 50/0...",de\tVerfahren zur Steuerung eines Kraftfahrzeu...,[Method for controlling a motor_vehicle_drive_...,[Verfahren zur Steuerung eines Kraftfahrzeug A...
2,EP03798697NWB1.xml,"[B29C 44/06 20060101AFI20121002BHEP, B60R 13/0...",de\tVERBESSERTE FAHRZEUGBODENBELÄGE UND VERFAH...,[A method of producing a vehicle_interior_floo...,[Verfahren zum Herstellen einer Innenbodenabde...
3,EP03815372NWB1.xml,"[B60J 5/06 20060101AFI20040810BHEP, B60J 5/04 ...",de\tSCHWENKSCHIEBETÜR FÜR FAHRZEUGE\ten\tPIVOT...,"[Plug door for vehicles , in particular for ra...","[Schwenkschiebetür für Fahrzeuge , insbesonder..."
4,EP05017320NWB1.xml,"[B60Q 3/04 20060101AFI20051116BHEP, B60K 37/00...",de\tKfz-Innenbeleuchtungsstruktur\ten\tVehicle...,[A vehicle_interior_illumination_structure_com...,"[Fahrzeuginnenraum Beleuchtungsstruktur , welc..."
...,...,...,...,...,...
995,EP08791706NWB1.xml,"[B60C 1/00 20060101AFI20121016BHEP, C08L 23/22...",de\tLuftreifen mit Schutzschicht\ten\tPneumati...,[A pneumatic_tire using a thermoplastic_elasto...,[Luftreifen unter Verwendung eines thermoplast...
996,EP08792532NWB1.xml,"[B60W 10/00 20060101AFI20090325BHEP, B60K 6/22...",de\tHYBRIDFAHRZEUG\ten\tHYBRID VEHICLE\tfr\tVÉ...,[A hybrid vehicle running by consuming a first...,"[Hybridfahrzeug , das unter Verwendung einer e..."
997,EP08794110NWB1.xml,"[B60T 1/08 20060101AFI20120919BHEP, B60W 10/18...",de\tBREMSSYSTEM UND FAHRZEUG MIT EINEM BREMSSY...,[A vehicle NUM having a powertrain and brake s...,[Fahrzeug NUM mit einem Antriebsstrang und ein...
998,EP08801238NWB1.xml,"[B64C 25/36 20060101AFI20120926BHEP, F16C 19/5...",de\tRADLAGERUNG EINES FLUGZEUGFAHRWERKS\ten\tW...,[],[]


In [344]:
epo_10_20_proc = []
for df in tq(epo_20xx):  
    epo_10_20_proc.extend(preprocess_claims(df))

epo_10_20_proc_df = pd.DataFrame(epo_10_20_proc)
epo_10_20_proc_df

,name,iprcs,titles,en_claims,de_claims
0,EP03703859NWB1.xml,"[B60L 11/12 20060101AFI20121031BHEP, B60L 11/1...",de\tMODULARER LEISTUNGSWANDLER MIT FLÜSSIGGEKÜ...,[A modular power converter comprising : a ther...,"[Modularer_Leistungswandler , der Folgendes um..."
1,EP04021666NWB1.xml,"[F16H 61/12 20100101AFI20121030BHEP, B60W 50/0...",de\tVerfahren zur Steuerung eines Kraftfahrzeu...,[Method for controlling a motor_vehicle_drive_...,[Verfahren zur Steuerung eines Kraftfahrzeug A...
2,EP03798697NWB1.xml,"[B29C 44/06 20060101AFI20121002BHEP, B60R 13/0...",de\tVERBESSERTE FAHRZEUGBODENBELÄGE UND VERFAH...,[A method of producing a vehicle_interior_floo...,[Verfahren zum Herstellen einer Innenbodenabde...
3,EP03815372NWB1.xml,"[B60J 5/06 20060101AFI20040810BHEP, B60J 5/04 ...",de\tSCHWENKSCHIEBETÜR FÜR FAHRZEUGE\ten\tPIVOT...,"[Plug door for vehicles , in particular for ra...","[Schwenkschiebetür für Fahrzeuge , insbesonder..."
4,EP05017320NWB1.xml,"[B60Q 3/04 20060101AFI20051116BHEP, B60K 37/00...",de\tKfz-Innenbeleuchtungsstruktur\ten\tVehicle...,[A vehicle_interior_illumination_structure com...,"[Fahrzeuginnenraum Beleuchtungsstruktur , welc..."
...,...,...,...,...,...
52666,EP16161642NWB1.xml,"[H02J 7/00 20060101AFI20160802BHEP, B60L 11/18...",de\tLADESTATION FÜR AUTONOM NAVIGIERENDES NUTZ...,[A charging station NUM for charging a battery...,[Ladestation NUM zum Laden einer Batterie NUM ...
52667,EP16162926NWB1.xml,"[B60N 2/00 20060101AFI20160601BHEP, A47D 7/03 ...",de\tTIEFENVERSTELLBARE WIEGE\ten\tDEPTH-ADJUST...,"[An adjustable depth bassinet NUM , comprising...",[Ein in der Tiefe verstellbarer Korbwagen NUM ...
52668,EP15713208NWB1.xml,"[H04L 29/08 20060101AFI20150923BHEP, G06F 9/44...",de\tVERFAHREN UND SYSTEME ZUM BESCHLEUNIGTEN H...,[A method for the downloading of data to at le...,[Verfahren zum Herunterladen von Daten auf min...
52669,EP16181157NWB1.xml,"[B60L 11/18 20060101AFI20170628BHEP, H01F 38/1...",de\tFAHRZEUG\ten\tVEHICLE\tfr\tVÉHICULE\t,[A vehicle NUM comprising a power_reception_de...,[Fahrzeug NUM mit einem Leistungsempfangsgerät...


In [334]:
words = []
for claims in tq(epo_10_20_proc_df.en_claims):
    for claim in claims:
        split = claim.split(' ')
        for i in split:
            words.append(i)
words = list(set(words))    

In [335]:
ind = 50
epo_20xx[0].en_claims[ind]

['Blank for a motor vehicle registration plate made from a metal sheet (14), the metal sheet (14) being partially reinforced by at least one multiple-ply region, characterized in that',
 'Blank according to Claim 1, characterized in that',
 'Blank according to Claim 1 or 2, characterized in that',
 'Blank according to Claim 1, characterized in that',
 'Blank according to Claim 1, characterized in that',
 'Motor vehicle registration plate with a metal sheet (14) and, if appropriate, with a reflex film (15) arranged on a front side (16) of the metal sheet (14), characterized by',
 'Method for producing a motor vehicle registration plate, in particular a blank for a motor vehicle registration plate, a cut-to-size piece being formed from a metal sheet (14), and the cut-to-size piece being provided, if appropriate, with a marginal embossing (10) and/or holes (17), characterized in that',
 'Method according to Claim 7, characterized in that',
 'Method according to Claim 7 or 8, characterized

In [336]:
epo_10_20_proc_df.en_claims[ind]

['Blank for a motor_vehicle_registration_plate made from a metal_sheet NUM , the metal_sheet NUM being partially reinforced by at least one multiple_ply_region , characterized in that',
 'Motor_vehicle_registration_plate with a metal_sheet NUM and , if appropriate , with a reflex_film NUM arranged on a front side NUM of the metal_sheet NUM , characterized by',
 'Method for producing a motor_vehicle_registration_plate , in particular a blank for a motor_vehicle_registration_plate , a cut_to_size_piece being formed from a metal sheet NUM , and the cut_to_size_piece being provided , if appropriate , with a marginal embossing NUM and or holes NUM , characterized in that']

In [337]:
epo_10_20_proc_df.de_claims[ind]

['Rohling für ein Kraftfahrzeugkennzeichen aus einem Blech NUM , wobei das Blech NUM durch mindestens einen mehrlagigen_Bereich teilweise verstärkt ist , dadurch gekennzeichnet , dass',
 'Kraftfahrzeugkennzeichen mit einem Blech NUM und gegebenenfalls einer auf einer Vorderseite NUM des Blechs NUM angeordneten Reflexfolie NUM , gekennzeichnet durch',
 'Verfahren zur Herstellung eines Kraftfahrzeugkennzeichens , insbesondere eines Rohlings für ein Kraftfahrzeugkennzeichen , wobei aus einem Blech NUM ein Zuschnitt gebildet wird und der Zuschnitt gegebenenfalls mit einer Randprägung NUM und oder Löchern NUM versehen wird , dadurch gekennzeichnet , dass']

In [225]:
# re_en_ncs = []
# for nc in tq(en_ncs):
#     nc = re.sub('-', '', nc)
#     nc = re.sub('\s{1,}', '_', nc)
#     re_en_ncs.append(nc)
#     
# re_de_ncs = []
# for nc in tq(de_ncs):
#     nc = re.sub('-', '', nc)
#     nc = re.sub('\s{1,}', '_', nc)
#     re_de_ncs.append(nc)

In [ ]:
# epo_18_20 = []
# for e, d in zip(tq(rep_en_claims), rep_de_claims):             
#     epo_18_20.append(
#         {
#             'en': e,
#             'de': d
#         }
#     )
# epo_18_20_df = pd.DataFrame(epo_18_20)
# epo_18_20_df

In [345]:
epo_10_20 = []
for name, titles, en, de in zip(epo_10_20_proc_df.name,
                                epo_10_20_proc_df.titles,
                                epo_10_20_proc_df.en_claims,
                                epo_10_20_proc_df.de_claims):
    for e, d in zip(en, de):
        epo_10_20.append(
            {
                'name': name,
                'titles': titles,
                'en': e,
                'de': d
            }
        )
epo_10_20_df = pd.DataFrame(epo_10_20)
epo_10_20_df

,name,titles,en,de
0,EP03703859NWB1.xml,de\tMODULARER LEISTUNGSWANDLER MIT FLÜSSIGGEKÜ...,A modular power converter comprising : a therm...,"Modularer_Leistungswandler , der Folgendes umf..."
1,EP03703859NWB1.xml,de\tMODULARER LEISTUNGSWANDLER MIT FLÜSSIGGEKÜ...,"The modular_power_converter of claim NUM , whe...","Modularer_Leistungswandler nach Anspruch NUM ,..."
2,EP03703859NWB1.xml,de\tMODULARER LEISTUNGSWANDLER MIT FLÜSSIGGEKÜ...,"The modular_power_converter of claim NUM , whe...","Modularer_Leistungswandler nach Anspruch NUM ,..."
3,EP03703859NWB1.xml,de\tMODULARER LEISTUNGSWANDLER MIT FLÜSSIGGEKÜ...,"The modular_power_converter of claim NUM , whe...","Modularer_Leistungswandler nach Anspruch NUM ,..."
4,EP03703859NWB1.xml,de\tMODULARER LEISTUNGSWANDLER MIT FLÜSSIGGEKÜ...,"The modular_power_converter of claim NUM , whe...","Modularer_Leistungswandler nach Anspruch NUM ,..."
...,...,...,...,...
365019,EP15713208NWB1.xml,de\tVERFAHREN UND SYSTEME ZUM BESCHLEUNIGTEN H...,A motor vehicle NUM such that it comprises at ...,"Kraftfahrzeug , das mindestens einen Computer ..."
365020,EP15713208NWB1.xml,de\tVERFAHREN UND SYSTEME ZUM BESCHLEUNIGTEN H...,A system for the downloading of data comprisin...,"System zum Herunterladen von Daten , das ein H..."
365021,EP16181157NWB1.xml,de\tFAHRZEUG\ten\tVEHICLE\tfr\tVÉHICULE\t,A vehicle NUM comprising a power_reception_dev...,Fahrzeug NUM mit einem Leistungsempfangsgerät ...
365022,EP16181157NWB1.xml,de\tFAHRZEUG\ten\tVEHICLE\tfr\tVÉHICULE\t,The vehicle NUM according to any one of claims...,Fahrzeug NUM nach einem der Ansprüche NUM bis ...


In [ ]:
# %%time
# epo_18_20_df.to_pickle('../../data/word2vec/epo/epo_18_20_df_wo_ncs.gzip', compression='gzip')

In [347]:
import sys
sys.path.append('../')
from util import util

In [348]:
%%time
util.compress_pickle('%s/pre_proc/epo_10_20_df_w_ncs'%path, epo_10_20_df)

CPU times: user 25.2 s, sys: 132 ms, total: 25.3 s
Wall time: 25.4 s


## Stop word removal and tokenization

In [19]:
import nltk
from nltk.corpus import stopwords

import string

from tqdm import tqdm as tq

In [20]:
stop_words = set(stopwords.words('english'))
stop_words_de = set(stopwords.words('german'))
punct = list(string.punctuation)

In [21]:
from nltk.stem import WordNetLemmatizer
lemmatizer_en = WordNetLemmatizer()

from nltk.stem.cistem import Cistem
lemmatizer_de = Cistem()

In [22]:
def get_tokens_nltk(text, lang='en', lemmatize=False):
    '''
    this function tokenizes and returns the unique tokens
    '''
    tokens = nltk.word_tokenize(text.lower())
    
    # assign the stopwords corresponding to the language
    if lang == 'en':
        stop_words_xx = stop_words
    elif lang == 'de':
        stop_words_xx = stop_words_de
    
    pre_proc_tokens = []
    for token in tokens:
        # check if the token is a digit or a stop word or a punctuation
        # eg: 1, is, und, '', etc.
        if token.isdigit() or token in stop_words_xx or token in punct:
            continue
            
        punct_found = False
        # check if there is a punctuation within the token
        # eg: vehic'le can be ignored
        for p in punct:
            if p in token:
                punct_found = True
                break
        if punct_found:
            continue
        
        # check if the token is a special token (in case of EPO claims)
        if token in ['[ABBRV]' , '[CLAIMS]']:
            continue
        
        if lemmatize:
            if lang == 'en': 
                lemma = lemmatizer_en.lemmatize(token)
            else:
                lemma = lemmatizer_de.stem(token)
            if lemma in pre_proc_tokens:
                continue
            pre_proc_tokens.append(lemma)
        else:
            if token in pre_proc_tokens:
                continue
            pre_proc_tokens.append(token)
            
    return pre_proc_tokens

In [198]:
corpus = epo_18_20_proc_df.en_claims.to_list()
corpus_de = epo_18_20_proc_df.de_claims.to_list()

In [199]:
vocab = []
all_tokens = []
for _ in tq(corpus):
    for text in _:
        # print(text)
        tokens = get_tokens_nltk(text, 'en', lemmatize=False)
        # print(tokens)
        vocab.extend(tokens)
        all_tokens.append(tokens)
vocab = list(set(vocab))
# print(vocab, end=' ')
print('\nLength', len(vocab))
print('\nLength all_tokens', len(all_tokens))


Length 30146

Length all_tokens 217865


In [200]:
vocab_de = []
all_tokens_de = []
for _ in tq(corpus_de):
    for text in _:
        tokens = get_tokens_nltk(text, 'de', lemmatize=False)
        vocab_de.extend(tokens)
        all_tokens_de.append(tokens)
vocab_de = list(set(vocab_de))
# print(vocab_de, end=' ')
print('\nLength', len(vocab_de))
print('\nLength all_tokens_de', len(all_tokens_de))


Length 172302

Length all_tokens_de 217867


In [208]:
# Save vocab and all_tokens 
file = open('epo_vocab_en.pkl', 'wb')
pickle.dump(vocab, file)
file.close()

file = open('epo_all_tokens_en.pkl', 'wb')
pickle.dump(all_tokens, file)9
file.close()


file = open('epo_vocab_de.pkl', 'wb')
pickle.dump(vocab_de, file)
file.close()

file = open('epo_all_tokens_de.pkl', 'wb')
pickle.dump(all_tokens_de, file)
file.close()

In [209]:
# Load vocab and all_tokens 
file = open('epo_vocab_en.pkl', 'rb')
vocab = pickle.load(file)
file.close()

file = open('epo_all_tokens_en.pkl', 'rb')
all_tokens = pickle.load(file)
file.close()


file = open('epo_vocab_de.pkl', 'rb')
vocab_de = pickle.load(file)
file.close()

file = open('epo_all_tokens_de.pkl', 'rb')
all_tokens_de = pickle.load(file)
file.close()

In [210]:
print('Vocab EN Length', len(vocab))
print('Vocab DE Length', len(vocab_de))
print('All tokens EN Length', len(all_tokens))
print('All tokens DE Length', len(all_tokens_de))

Vocab EN Length 30146
Vocab DE Length 172302
All tokens EN Length 217865
All tokens DE Length 217867
